In [1]:
# !pip install selenium
# !pip install django
# !pip install django-embed-video

In [1]:
import csv
import uuid
from selenium import webdriver
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd

import requests
import urllib
import os

import numpy as np

In [3]:
# ----------------- Crawl data -----------------
titles  = []
years   = []
genres  = []
plots   = []
posters = []
ratings = []
trailors= []

data=[]
link_films = []
# ---------------------------------------------------

In [ ]:
s=Service('C:/Users/Van Thong/Downloads/chromedriver.exe')
driver = webdriver.Chrome(service=s) 
for i in range(1, 501):
    driver.get('https://flixgo.up.railway.app/movies-catalog/')
    your_input = driver.find_element(By.XPATH,'//div//ul/form[@method="post"]//input[@id="page"]')
    your_input.clear()
    your_input.send_keys(str(i))
    your_input.send_keys(Keys.ENTER)
    link_film=driver.find_elements(By.XPATH,"//div//a[@class='card__play']")
    for link in link_film:
        link = link.get_attribute("href")
        link_films.append({"link":link})
driver.quit()
print(link_films)

In [ ]:
for i in link_films:
    driver = webdriver.Chrome(service=s)
    driver.get(i["link"])
    
    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(1) > span').forEach(el => el.remove());");

    driver.execute_script("document.querySelectorAll('body > section.section.details > div:nth-child(3) > div.row > div:nth-child(2) > div > div > div.col-12.col-sm-8.col-md-8.col-lg-9.col-xl-7 > div > ul > li:nth-child(2) > span').forEach(el => el.remove());");

    # Lấy tên phim
    title = driver.find_element(By.XPATH,"//h1[@class='details__title']").text
   
    # ------------------Lấy năm phát hành------------------
    try:
        year = driver.find_element(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li[2]").text
        # Chỉ lấy mỗi năm
        year = year.split("-")[0]
    except:
        year = ''
    years.append({'year': year})
    # ------------------------------------------------------

    # ---------------------Lấy thể loại phim---------------------
    genre = driver.find_elements(By.XPATH,"/html/body/section[1]/div[3]/div[1]/div[2]/div/div/div[2]/div/ul/li/a")
    temp_genres={'genre':[]}

    for i in genre:
        temp_genres['genre'].append(i.text)
    genres_str = ', '.join(temp_genres['genre'])
    new_dict = {'genre': genres_str}
    genres.append(new_dict)
    # --------------------------------------------------------------

    # Lấy Overview
    try:
        plot = driver.find_element(By.XPATH,"//div[@class='card__description card__description--details']").text
    except:
        plot = ''
    # Lấy đánh giá
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='card__rate']").text
    except:
        rating = ''
    # Lấy link poster 
    driver.execute_script("window.scrollTo(0, 990000)")
    poster = driver.find_element(By.XPATH,"//div[@class='card__cover']//img").get_attribute("src")
    
    # Lấy link trailer link youtobe
    # Nếu trailer không có thì sẽ trả về rổng
    try:
        trailor = driver.find_element(By.XPATH,"//div[@class='col-12 col-xl-6']//iframe").get_attribute("src")
    except:
        trailor = None

    # thêm vào list các dict tương ứng
    titles.append({'title': title})
    ratings.append({'rating': rating})
    posters.append({'poster': poster})
    plots.append({'plot': plot})
    trailors.append({'trailor': trailor})

    driver.close()

In [ ]:
# Lưu dữ liệu trên vào file csv
with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'year', 'genre', 'plot', 'poster', 'rating', 'trailor'])
    for i in range(len(titles)):
        writer.writerow([titles[i]['title'], years[i]['year'], genres[i]['genre'], plots[i]['plot'], posters[i]['poster'], ratings[i]['rating'], trailors[i]['trailor']])


In [1]:
import pandas as pd

In [3]:
# Đọc file csv
df = pd.read_csv('data_done.csv', encoding='utf-8')
len(df)

8950

In [5]:
# Lọc các dữ liệu trùng lặp
df = df.drop_duplicates(subset=['title'])
len(df)

8655

In [6]:
# Lưu dữ liệu vào file csv
df.to_csv('data_done_NEW.csv', index=False)


In [7]:
# Lọc các title có len trên 100 ký tự
df = df[df['title'].str.len() < 100]
len(df)

8654

In [8]:
df.to_csv('data_done_NEW_1.csv', index=False)

`Chuyển đổi kiểu dữ liệu của cột 'year' sang kiểu số nguyên (integer)`

In [7]:
df = df.dropna(subset=['year'])
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['year'])

df['year'] = df['year'].astype(int) 

`Kiểm tra df có giá trị trùng không xóa các dữ liệu trùng đó`


In [4]:
df.duplicated(subset=['title', 'year']).sum()
df = df.drop_duplicates(subset=['title', 'year'], keep='first')
len(df)

8950